# QC taxa

In [2]:
import sys
sys.path.append('../../')
import pandas as pd
import glob
from pathlib import Path
from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR
import db as db


from scripts.normalize_data import (
    csv_cleanup,
    update_metadata,
    get_taxonomy_columns,
)

import scripts.normalize_taxa as nt

from scripts.shared_utils import (
    log_df
)

In [3]:
clean_data_paths = [
    CLEAN_DATA_DIR /'LIMS/Micropal_CSV_1', 
    CLEAN_DATA_DIR /'LIMS/Micropal_CSV_2', 
    CLEAN_DATA_DIR /'LIMS/Micropal_CSV_3', 
    CLEAN_DATA_DIR /'LIMS/Micropal_CSV_4', 
    CLEAN_DATA_DIR /'LIMS/Micropal_CSV_revised', 
]

metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes.csv' 


date = '2022-08-08'
taxa_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_list_{date}.csv"
taxa_crosswalk_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_crosswalk_{date}.csv"

date = '2021-11-29'
noaa_taxa_crosswalk_file = OUTPUT_DIR/'taxa'/'NOAA'/f"PI_normalized_taxa_list_with_pbdb_{date}.csv"


date = '2022-08-08'
input_file = RAW_DATA_DIR/'PI_processed_files'/f'LIMS_Micropal_headers_PBDB_Taxonomy_notes_taxa_list_{date}.csv'
input_4_file = RAW_DATA_DIR/'PI_processed_files'/f'LIMS Micropal_CSV_4_taxa_{date}.csv'

pi_file = OUTPUT_DIR/'taxa'/'LIMS'/'PI_normalized_taxa_list_with_pbdb_2022-04-28.csv'

date='2021-11-29'
input_noaa_file = RAW_DATA_DIR/'PI_processed_files'/f'NOAA_taxa_lists_taxa_list_{date}.csv'

add_taxa_file = RAW_DATA_DIR/'PI_processed_files'/'LIMS_Micropal_CSV_4_taxa_ADDTL_TAXA.csv'

date = '2022-02-24'
taxa_file = OUTPUT_DIR/'taxa'/'draft'/'LIMS'/f"micropal_4_taxa_{date}.csv"
genus_file= OUTPUT_DIR/'taxa'/'draft'/'LIMS'/f"micropal_4_genus_{date}.csv"
taxa_pbdb_file = OUTPUT_DIR/'taxa'/'draft'/'LIMS'/f'micropal_4_taxa_pbdb_{date}.csv'
genus_letter_file= OUTPUT_DIR/'taxa'/'draft'/'LIMS'/f"micropal_4_genus_letter_{date}.csv"

all_columns_file = OUTPUT_DIR/'tmp/all_LIMS_taxa_columns.csv'


date = '2022-08-30'
nontaxa_fields_path =  OUTPUT_DIR/'normalized_data'/'LIMS'/f'nontaxa_fields_{date}.csv'

PI_abundance_codes_group_path = RAW_DATA_DIR/'PI_processed_files'/'group_abundance_preservation.csv'


# compare taxa list and PI taxa list

Check if the taxa that were aproved by the researchers from the google sheet matches the taxa from the LIMS taxa_list.csv.

In [188]:
normalized_df = pd.read_csv(input_file, skiprows = 9)
normalized_df = normalized_df.dropna(axis=0, how='all')
normalized_df.shape

(4743, 32)

In [192]:
normalized_4_df = pd.read_csv(input_4_file)
normalized_4_df = normalized_4_df.dropna(axis=0, how='all')
normalized_4_df.shape

(695, 33)

In [193]:
taxa_df = pd.read_csv(taxa_crosswalk_list_file)
taxa_df = taxa_df.dropna(axis=0, how='all')
taxa_df.shape

(5380, 18)

In [203]:
noaa_df = pd.read_csv(input_noaa_file)
noaa_df = noaa_df.dropna(axis=0, how='all')
noaa_df.shape

(7763, 20)

In [201]:
normalized_names = set(normalized_df['verbatim_name'])
normalized_4_names = set(normalized_4_df['verbatim_name'])
all_normalized_names = normalized_names.union(normalized_4_names)

taxa_names = set(taxa_df['verbatim_name'])
len(taxa_names)

5368

In [205]:
names = all_normalized_names - taxa_names
len(names)

28

In [206]:
len(names) == 28

True

In [207]:
names

{'ADDITIONAL SPECIES',
 'Bathymetry',
 'Dextral:Sinistral _N. acostaensis_',
 'Dextral:Sinistral _P. finalis_',
 'Dextral:Sinistral _P. obliquiloculata_',
 'Dextral:Sinistral _P. praecursor_',
 'Dextral:Sinistral _P. praespectabilis_',
 'Dextral:Sinistral _P. primalis_',
 'Dextral:Sinistral _P. spectabilis_',
 'Diatom Zone (NPD) in Yanagisawa and Akiba (1998)',
 'Diatom Zone (Yanagisawa and Akiba, 1998)',
 'Exotic',
 'Gen. et sp. indet',
 'Marine',
 'Martini (1971) Zone',
 'Organic matter',
 'Planktic foraminiferal %',
 'Planktic foraminiferal (%)',
 'Preservation palynofacies',
 'Pyrite',
 'RESOLVED NAMES BY TAXONOMIC GROUP',
 'Radiolarian zone',
 'Radiolarian zone/subzone',
 'Silicoflagellate Zone in Ling (1992)',
 'Terrestrial organic matter',
 'Zone in Ling (1992)',
 'fossil',
 'fossil_group'}

In [208]:
names = taxa_names - all_normalized_names - set(noaa_df['verbatim_name'])

len(names)

34

In [210]:
len(names) == 34

True

In [209]:
names

{'Dextral N. acostaensis',
 'Dextral P. finalis',
 'Dextral P. obliquiloculata',
 'Dextral P. praecursor',
 'Dextral P. praespectabilis',
 'Dextral P. primalis',
 'Dextral P. spectabilis',
 'Sinistral N. acostaensis',
 'Sinistral P. finalis',
 'Sinistral P. obliquiloculata',
 'Sinistral P. praecursor',
 'Sinistral P. praespectabilis',
 'Sinistral P. primalis',
 'Sinistral P. spectabilis',
 'additional species: Big Lithomelissa sp. A (Antarctissa like)',
 'additional species: Cypassis irregularis',
 'additional species: Eucyrtidium teuscheri',
 'additional species: Gondwanaria dogieli',
 'additional species: Grobolotalia spp.',
 'additional species: Hantkenina australis',
 'additional species: Larcopyle buetschlii (R)',
 'additional species: Litharachnium tentorium',
 'additional species: Lithostrobus cuspidatus',
 'additional species: Lonchosphaera spicata',
 'additional species: N. incompta',
 'additional species: Neogloboquadrina pachyderma',
 'additional species: Parasubbotina varia

# check if taxon with multiple taxon groups is imported

'Globigerinoides ruber (white)' has both benthic_forams; planktic_forams

In [112]:
sql = f"""select 
taxa.name as taxon_name,
taxa.taxon_group,
 samples.data_source_notes
from samples
join samples_taxa on samples_taxa.sample_id = samples.id
join taxa on samples_taxa.taxon_id = taxa.id
where  samples.data_source_type = 'micropal csv'
and taxa.name = 'Globigerinoides ruber (white)';"""

rows = db.fetch_all(sql)
data = []
for row in rows:
    data.append({
        'verbatim_name': row['taxon_name'],
        'taxon_group': row['taxon_group'],
        'data_source_notes': row['data_source_notes']
    })
    

db_df = pd.DataFrame(data)
db_df.drop_duplicates(inplace=True, subset=['verbatim_name', 'taxon_group' ])
db_df

,verbatim_name,taxon_group,data_source_notes
0,Globigerinoides ruber (white),planktic_forams,LIMS/Micropal_CSV_2/368_U1505D_planktic_forams...
1134,Globigerinoides ruber (white),benthic_forams,LIMS/Micropal_CSV_2/356-U1463B_benthic_forams.csv


In [114]:
list(db_df['verbatim_name'])  == ['Globigerinoides ruber (white)', 'Globigerinoides ruber (white)']

True

In [115]:
list(db_df['taxon_group'])  == ['planktic_forams', 'benthic_forams']

True

# check  dextral sinistral import

In [13]:
file = 'LIMS/Micropal_CSV_1/363-U1482A-planktic_forams.csv'

In [14]:

sql = f"""select samples.name as sample_name,  samples.eodp_id,
taxa_crosswalk.verbatim_name as verbatim_name,
taxa.name as taxon_name,
samples_taxa.code
from samples_taxa
join samples on samples.id = samples_taxa.sample_id
join taxa on samples_taxa.taxon_id = taxa.id

join taxa_crosswalk on taxa_crosswalk.id = samples_taxa.original_taxon_id

where samples_taxa.data_source_notes = '{file}';"""

rows = db.fetch_all(sql)
data = []
for row in rows:
    data.append({
        'Sample': row['sample_name'], 
        'verbatim_name': row['verbatim_name'],
        'code': row['code'],
        'eodp_id': row['eodp_id']
    })

db_df = pd.DataFrame(data)
log_df(db_df)

(478, 4)


,Sample,verbatim_name,code,eodp_id
0,363-U1482A-1H-CC-PAL-FORAM,Globigerinella calida _B_,R,581cf8d872bc892b3d7664f3c868745f
1,363-U1482A-1H-CC-PAL-FORAM,Globorotalia excelsa _B_,P,581cf8d872bc892b3d7664f3c868745f
2,363-U1482A-1H-CC-PAL-FORAM,Pulleniatina finalis _B,P,581cf8d872bc892b3d7664f3c868745f
3,363-U1482A-1H-CC-PAL-FORAM,Globorotalia truncatulinoides _B_,R,581cf8d872bc892b3d7664f3c868745f
4,363-U1482A-1H-CC-PAL-FORAM,Pulleniatina obliquiloculata,F,581cf8d872bc892b3d7664f3c868745f


In [15]:
db_pivot = db_df.pivot(index=['Sample', 'eodp_id'], columns='verbatim_name', values='code')
db_pivot = db_pivot.reset_index()
 
db_pivot.sort_values(['Sample', 'eodp_id'], inplace=True)

log_df(db_pivot)

(71, 73)


verbatim_name,Sample,eodp_id,"""Globigerina"" angulisuturalis _T",Candeina nitida,Candeina praenitida,Cassigerinella chipolensis _T,"Dentoglobigerina ""conglomerata""",Dentoglobigerina altispira _T_ _PL5,Dentoglobigerina baroemoenensis,Dentoglobigerina binaensis _T_,...,Sinistral P. praespectabilis,Sinistral P. primalis,Sphaeroidinella dehiscens sensu lato _B_,Sphaeroidinella excavata,Sphaeroidinellopsis kochi _T_,Sphaeroidinellopsis paenedehiscens,Sphaeroidinellopsis seminulina _T_ _PL4,Trilobatus bisphericus,Trilobatus sacculifer,Trilobatus trilobus _B
0,363-U1482A-10H-CC-PAL-FORAM,f7fa5407f5c0964abf1fd3da02fbd441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,363-U1482A-11H-CC-PAL-FORAM,d3af6afade68362574996c493140a750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,363-U1482A-12H-CC-PAL-FORAM,70b431e20011c6451b591d4b525b6a6d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,363-U1482A-13H-CC-PAL-FORAM,eea9bab5737b68b7b5029aa30781645d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,363-U1482A-14H-CC-PAL-FORAM,ee887c246f7318a6f17a0d83d927004d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
file_df = pd.read_csv(CLEAN_DATA_DIR/file, dtype=str)
log_df(file_df, 2)

(71, 251)


,Sample,Top [cm],Bottom [cm],Top Depth [m],Bottom Depth [m],Zone name (short),Zone name,Additional zone name (short),Additional zone name,Preservation,...,Dextral P. obliquiloculata,Sinistral P. obliquiloculata,Dextral P. primalis,Sinistral P. primalis,Dextral P. praecursor,Sinistral P. praecursor,Dextral P. spectabilis,Sinistral P. spectabilis,Dextral P. finalis,Sinistral P. finalis
0,363-U1482A-1H-CC-PAL-FORAM,0,5,2.93,2.98,PT1b,PT1b - Globorotalia truncatulinoides Partial-r...,NaN,NaN,E [P46],...,49,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,363-U1482A-2H-CC-PAL-FORAM,0,5,12.7,12.75,PT1b,PT1b - Globorotalia truncatulinoides Partial-r...,NaN,NaN,E [P46],...,15,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
taxa_cols = set(db_pivot.columns) - {'Sample'}


filter_df = file_df[db_pivot.columns].copy()
filter_df.dropna(axis=0, how='all', inplace=True, subset=taxa_cols)
filter_df.dropna(axis=1, how='all', inplace=True)

filter_df.sort_values(['Sample', 'eodp_id'], inplace=True)

log_df(filter_df, 2)

(71, 73)


,Sample,eodp_id,"""Globigerina"" angulisuturalis _T",Candeina nitida,Candeina praenitida,Cassigerinella chipolensis _T,"Dentoglobigerina ""conglomerata""",Dentoglobigerina altispira _T_ _PL5,Dentoglobigerina baroemoenensis,Dentoglobigerina binaensis _T_,...,Sinistral P. praespectabilis,Sinistral P. primalis,Sphaeroidinella dehiscens sensu lato _B_,Sphaeroidinella excavata,Sphaeroidinellopsis kochi _T_,Sphaeroidinellopsis paenedehiscens,Sphaeroidinellopsis seminulina _T_ _PL4,Trilobatus bisphericus,Trilobatus sacculifer,Trilobatus trilobus _B
9,363-U1482A-10H-CC-PAL-FORAM,f7fa5407f5c0964abf1fd3da02fbd441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,363-U1482A-11H-CC-PAL-FORAM,d3af6afade68362574996c493140a750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
db_pivot.to_csv(OUTPUT_DIR/'tmp'/'db_pivot.csv', index=False)
filter_df.to_csv(OUTPUT_DIR/'tmp'/'filter_df.csv', index=False)

In [19]:
dfa = pd.read_csv(OUTPUT_DIR/'tmp'/'db_pivot.csv', dtype=str)
dfb = pd.read_csv(OUTPUT_DIR/'tmp'/'filter_df.csv', dtype=str)

dfa.equals(dfb)

True

# check  files with multiple taxon groups is imported

'Globigerinoides ruber (white)' has both benthic_forams; planktic_forams

file has benthic_forams and planktic_forams

In [20]:
file = 'LIMS/Micropal_CSV_2/356-U1463B_benthic_forams.csv'

In [21]:

sql = f"""select samples.name as sample_name, 
taxa_crosswalk.verbatim_name as taxon_name,
 samples_taxa.code, samples_taxa.data_source_notes,
taxa.taxon_group
from samples_taxa
join samples on samples.id = samples_taxa.sample_id
join taxa_crosswalk on taxa_crosswalk.id = samples_taxa.original_taxon_id
join taxa on samples_taxa.taxon_id = taxa.id

where samples_taxa.data_source_notes = '{file}';"""

rows = db.fetch_all(sql)
data = []
for row in rows:
    data.append({
        'Sample': row['sample_name'], 
        'verbatim_name': row['taxon_name'],
        'code': row['code'],
    })

db_df = pd.DataFrame(data)
db_df.head()

,Sample,verbatim_name,code
0,356-U1463B-1H-CC-PAL-FORAM-150-2000,Globigerinella siphonifera,R
1,356-U1463B-1H-CC-PAL-FORAM-150-2000,Globorotalia tumida,R
2,356-U1463B-1H-CC-PAL-FORAM-150-2000,Globigerinoides conglobatus,R
3,356-U1463B-1H-CC-PAL-FORAM-150-2000,Globigerinoides sacculifer,R
4,356-U1463B-1H-CC-PAL-FORAM-150-2000,Pulleniatina obliquiloculata,F


In [22]:
db_pivot = db_df.pivot(index='Sample', columns='verbatim_name', values='code')
db_pivot = db_pivot.reset_index()

 
db_pivot.sort_values('Sample', inplace=True)

log_df(db_pivot)

(24, 48)


verbatim_name,Sample,Candeina nitida,Dentoglobigerina altispira,Dentoglobigerina baroemoenensis,Globigerina bulloides,Globigerina rubescens,Globigerinella calida,Globigerinella siphonifera,Globigerinita glutinata,Globigerinoides bulloideus,...,Neogloboquadrina acostaensis,Neogloboquadrina dutertrei,Neogloboquadrina humerosa,Orbulina bilobata,Orbulina universa,Pulleniatina obliquiloculata,Pulleniatina primalis,Sphaeroidinella dehiscens,Sphaeroidinellopsis kochi,Sphaeroidinellopsis seminulina
0,356-U1463B-13H-CC-PAL-FORAM-150 2000,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,NaN,...,NaN,P,NaN,NaN,P,F,NaN,NaN,NaN,NaN
1,356-U1463B-15H-CC-PAL-FORAM-150-2000,NaN,NaN,NaN,P,NaN,NaN,P,NaN,NaN,...,NaN,P,NaN,NaN,P,P,NaN,NaN,NaN,NaN
2,356-U1463B-17H-CC-PAL-FORAM-150 2000,NaN,NaN,NaN,P,NaN,NaN,NaN,P,NaN,...,NaN,P,NaN,NaN,NaN,P,NaN,NaN,NaN,NaN
3,356-U1463B-19H-6-PAL-FORAM-150-2000,NaN,NaN,NaN,P,NaN,P,P,NaN,NaN,...,NaN,A,NaN,NaN,P,P,NaN,NaN,NaN,NaN
4,356-U1463B-1H-CC-PAL-FORAM-150-2000,P,NaN,NaN,NaN,R,P,R,NaN,NaN,...,NaN,R,NaN,NaN,R,F,NaN,P,NaN,NaN


In [23]:
file_df = pd.read_csv(CLEAN_DATA_DIR/file)
log_df(file_df, 2)

(32, 174)


,Sample,Top [cm],Bottom [cm],Top Depth [m],Bottom Depth [m],Datum name,Datum name generic,Datum comment,Datum age average [Ma],Zone name (short),...,Shore File Links,File Data,Exp,Site,Hole,Core,Type,Section,A/W,eodp_id
0,356-U1463B-1H-CC-PAL-FORAM-150-2000,0,10,7.78,7.88,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,356,U1463,B,1,H,CC,PAL,8adfb3c6f33088d72a886ed17287a589
1,356-U1463B-3H-CC-PAL-FORAM-150-2000,0,10,27.10,27.20,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,356,U1463,B,3,H,CC,PAL,93cc7d07dca008920dac6892a7489d30


In [24]:
taxa_cols = set(db_pivot.columns) - {'Sample'}


filter_df = file_df[db_pivot.columns].copy()
filter_df.dropna(axis=0, how='all', inplace=True, subset=taxa_cols)
filter_df.dropna(axis=1, how='all', inplace=True)

filter_df.sort_values('Sample', inplace=True)

log_df(filter_df, 2)

(24, 48)


,Sample,Candeina nitida,Dentoglobigerina altispira,Dentoglobigerina baroemoenensis,Globigerina bulloides,Globigerina rubescens,Globigerinella calida,Globigerinella siphonifera,Globigerinita glutinata,Globigerinoides bulloideus,...,Neogloboquadrina acostaensis,Neogloboquadrina dutertrei,Neogloboquadrina humerosa,Orbulina bilobata,Orbulina universa,Pulleniatina obliquiloculata,Pulleniatina primalis,Sphaeroidinella dehiscens,Sphaeroidinellopsis kochi,Sphaeroidinellopsis seminulina
5,356-U1463B-13H-CC-PAL-FORAM-150 2000,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,NaN,...,NaN,P,NaN,NaN,P,F,NaN,NaN,NaN,NaN
6,356-U1463B-15H-CC-PAL-FORAM-150-2000,NaN,NaN,NaN,P,NaN,NaN,P,NaN,NaN,...,NaN,P,NaN,NaN,P,P,NaN,NaN,NaN,NaN


In [25]:
db_pivot.to_csv(OUTPUT_DIR/'tmp'/'db_pivot.csv', index=False)
filter_df.to_csv(OUTPUT_DIR/'tmp'/'filter_df.csv', index=False)

In [26]:
dfa = pd.read_csv(OUTPUT_DIR/'tmp'/'db_pivot.csv', dtype=str)
dfb = pd.read_csv(OUTPUT_DIR/'tmp'/'filter_df.csv', dtype=str)

dfa.equals(dfb)

True

# check taxa names were imported into database

In [119]:
df = pd.read_csv(taxa_list_file, usecols=['normalized_name', 'taxon_group'])
df.drop_duplicates(subset=['normalized_name', 'taxon_group'], inplace=True)
df.shape
# 4738

(4738, 2)

In [120]:
sql = """
SELECT count(*) FROM taxa

"""
db.fetch_one(sql)

[4738]

In [121]:
len(df) == db.fetch_one(sql)[0]

True

In [122]:
df2 = pd.read_csv(taxa_crosswalk_list_file, usecols=['normalized_name', 'taxon_group', 'verbatim_name', 'eodp_id' ])
df2.shape
# 5380

(5380, 4)

In [123]:
sql = """
SELECT * FROM taxa_crosswalk 
JOIN taxa on taxa.id = taxa_crosswalk.taxon_id 

"""
rows = db.fetch_all(sql)
data = []
for row in rows:
    data.append({
        'normalized_name': row['name'], 
        'taxon_group': row['taxon_group'],
        'verbatim_name': row['verbatim_name'],
        'eodp_id': row['eodp_id']
        
    })


len(data)

5380

In [124]:
len(df2) == len(data)

True

In [125]:
db_df = pd.DataFrame(data)
db_df.head()

,normalized_name,taxon_group,verbatim_name,eodp_id
0,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,0
1,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,1
2,Foraminifera indet.,benthic_forams,Others,2
3,Pleurostomellidae indet.,benthic_forams,Pleurostomellids comment,3
4,Ostracoda indet.,benthic_forams,Ostracoda spp.,4


In [126]:
set(df2['eodp_id']) - set (db_df['eodp_id'])

set()

In [127]:
set(db_df['eodp_id']) - set (df2['eodp_id'])

set()

# check additional species are imported

In [175]:
file = 'LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv'
path = CLEAN_DATA_DIR/file
df = pd.read_csv(path, dtype=str)
df = df.dropna(axis=1, how="all")

log_df(df)

(31, 77)


,Sample,Top [cm],Bottom [cm],Top Depth [m],Bottom Depth [m],Zone name,Preservation,Group Abundance,Antarctissa cylindrica,Cycladophora pliocenica,...,additional species: Gondwanaria dogieli,additional species: Tetraplecta pinigera/Euscenium corynephorum,additional species: Litharachnium tentorium,additional species: Streblacantha circumtexta,additional species: Eucyrtidium teuscheri,additional species: Verticillata hexacantha,additional species: Cypassis irregularis,additional species: Prunopyle antarctica,additional species: Big Lithomelissa sp. A (Antarctissa like),additional species: Larcopyle buetschlii (R)
0,374-U1525A-1H-1-IW_MUDLINE,0,0,0,0,NaN,G,A,NaN,NaN,...,Ind,Ind,Ind,Ind,Ind,Ind,Ind,Ind,NaN,NaN
1,374-U1525A-1H-CC-PAL-RADS,0,5,8.57,8.62,NaN,P,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,374-U1525A-2H-CC-PAL-RADS,0,5,18.72,18.77,NaN,P,Tr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,374-U1525A-3H-CC-PAL-RADS,0,5,28.16,28.21,NaN,P,Tr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,374-U1525A-4H-CC-PAL-RADS,0,5,28.86,28.91,> 0.65 (LAD A. cylindrica),M,Tr,X,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
df.columns

Index(['Sample', 'Top [cm]', 'Bottom [cm]', 'Top Depth [m]',
       'Bottom Depth [m]', 'Zone name', 'Preservation', 'Group Abundance',
       'Antarctissa cylindrica', 'Cycladophora pliocenica',
       'Triceraspyris antarctica', 'Eucyrtidium calvertense',
       'Helotholus vema', 'Desmospyris spongiosa', 'Cycladophora davisiana',
       'Ceratocyrtis mashae', 'Prunopyle hayesi', 'Actinomma popofskii',
       'Acrosphaera? mercurius', 'Actinomma boreale', 'Actinomma delicatulum',
       'Actinomma leptodermum', 'Actinomma leptodernum longispinum',
       'Antarctissa denticulata', 'Antarctissa strelkovi',
       'Cenosphaera cristata', 'Ceratocyrtis spp.', 'Cornutella profunda',
       'Cycladophora bicornis', 'Druppatractus hastatus',
       'Enneaphormis rotula', 'Eucyrtidium inflatum',
       'Hexacontium pachydermum', 'Larcopyle pylomaticus',
       'Larcopyle weddellium', 'Lithelius nautiloides', 'Lithelius sp. A',
       'Mitrocalpis araneafera', 'Peripyramis circumtexta',
    

In [160]:
cols = [
    'Antarctissa cylindrica', 'Cycladophora pliocenica',
    'Triceraspyris antarctica', 'Eucyrtidium calvertense',
    'Helotholus vema', 'Desmospyris spongiosa', 'Cycladophora davisiana',
    'Ceratocyrtis mashae', 'Prunopyle hayesi', 'Actinomma popofskii',
    'Acrosphaera? mercurius', 'Actinomma boreale', 'Actinomma delicatulum',
    'Actinomma leptodermum', 'Actinomma leptodernum longispinum',
    'Antarctissa denticulata', 'Antarctissa strelkovi',
    'Cenosphaera cristata', 'Ceratocyrtis spp.', 'Cornutella profunda',
    'Cycladophora bicornis', 'Druppatractus hastatus',
    'Enneaphormis rotula', 'Eucyrtidium inflatum',
    'Hexacontium pachydermum', 'Larcopyle pylomaticus',
    'Larcopyle weddellium', 'Lithelius nautiloides', 'Lithelius sp. A',
    'Mitrocalpis araneafera', 'Peripyramis circumtexta',
    'Phormacantha hystrix/Plectacantha oikiskos group',
    'Phormostichoartus corbula', 'Pseudodictyophimus gracilipes',
    'Saccospyris antarctica', 'Saccospyris conithorax',
    'Saccospyris praeantarctica', 'Spongopyle osculosa',
    'Spongotrochus glacialis', 'Spongotrochus sp. A Abelmann',
    'Sphaeropyle robusta', 'Prunopyle tetrapila', 'Stylatractus neptunus',
    'Stylochlamidium  venustum', 'Stylodictya spp.', 'Trisulcus nana',
    # 'ADDITIONAL SPECIES', 
]

add_cols = [
    'additional species: Lonchosphaera spicata',
    'additional species: Poulpus spp.-like',
    'additional species: Lithostrobus cuspidatus',
    'additional species: Gondwanaria dogieli',
    'additional species: Tetraplecta pinigera/Euscenium corynephorum',
    'additional species: Litharachnium tentorium',
    'additional species: Streblacantha circumtexta',
    'additional species: Eucyrtidium teuscheri',
    'additional species: Verticillata hexacantha',
    'additional species: Cypassis irregularis',
    'additional species: Prunopyle antarctica',
    'additional species: Big Lithomelissa sp. A (Antarctissa like)',
    'additional species: Larcopyle buetschlii (R)'
]

In [177]:
data = []
for index, row in df.iterrows():
    for col in cols:
        if pd.notna(row[col]):
            data.append({'Sample': row['Sample'], 'code': row[col], 'taxon': col})
 
len(data)

114

In [178]:
sql = f"""
SELECT count(*) 
FROM  samples_taxa
WHERE (data_source_notes = '{file}')
AND (code != 'Ind') 
"""

count = db.fetch_one(sql)[0]
count

115

In [180]:
len(data) + 1 == count

True

In [183]:
data = []
for index, row in df.iterrows():
    for col in add_cols:
        if pd.notna(row[col]):
            data.append({'Sample': row['Sample'], 'code': row[col], 'taxon': col})
 
len(data)

17

In [184]:
sql = f"""
SELECT count(*) 
FROM  samples_taxa
WHERE (data_source_notes = '{file}')
AND (code = 'Ind') 
"""

count = db.fetch_one(sql)[0]
count

16

In [186]:
len(data) - 1 == count

True

# check for taxa in multiple taxon groups

In [52]:
# https://stackoverflow.com/questions/38127209/how-to-use-groupby-to-concatenate-strings-in-python-pandas

def check_taxa_by_groups(path, name_col):
    taxa_df = pd.read_csv(path)
    nt.add_normalized_name_column(taxa_df)
    taxa_df = taxa_df.drop_duplicates(subset=[name_col,  'taxon_group'])

    data = taxa_df.groupby([name_col])['taxon_group'].apply('; '.join)
    grouped_df = pd.DataFrame(data)

    grouped_df = grouped_df[grouped_df['taxon_group'].str.contains('; ')]
    grouped_df.reset_index(inplace=True)
    
    return grouped_df


In [147]:
df = check_taxa_by_groups(pi_file, 'verbatim_name')
df

,verbatim_name,taxon_group
0,ADDITIONAL SPECIES,planktic_forams; radiolarians
1,Amorphous organic matter,dinoflagellates; palynology
2,Black phytoclasts,dinoflagellates; palynology
3,Brown phytoclasts,dinoflagellates; palynology
4,Dinocysts,dinoflagellates; palynology
5,Globigerinoides ruber (white),benthic_forams; planktic_forams
6,Neogloboquadrina humerosa,benthic_forams; planktic_forams


In [149]:
len(df) == 7

True

In [150]:
df = check_taxa_by_groups(taxa_crosswalk_list_file, 'verbatim_name')
df

,verbatim_name,taxon_group
0,Amorphous organic matter,dinoflagellates; palynology
1,Black phytoclasts,dinoflagellates; palynology
2,Brown phytoclasts,dinoflagellates; palynology
3,Dinocysts,dinoflagellates; palynology
4,Echinoid plate fragments,other; planktic_forams
5,Echinoid spines,other; planktic_forams
6,Fish teeth,other; planktic_forams
7,Globigerinoides ruber (white),benthic_forams; planktic_forams
8,Neogloboquadrina humerosa,benthic_forams; planktic_forams
9,Ostracods,other; planktic_forams


In [151]:
len(df) == 12

True

In [152]:
df = check_taxa_by_groups(pi_file, 'normalized_name')
df

,normalized_name,taxon_group
0,,benthic_forams; palynology; planktic_forams; r...
1,"""Amorphous organic matter""",dinoflagellates; palynology
2,"""Black phytoclasts""",dinoflagellates; palynology
3,"""Brown phytoclasts""",dinoflagellates; palynology
4,Brigantedinium spp.,dinoflagellates; palynology
5,Dinoflagellata indet.,dinoflagellates; nannofossils; palynology
6,Foraminifera indet.,benthic_forams; dinoflagellates; palynology; p...
7,Globigerinoides ruber (white),benthic_forams; planktic_forams
8,Lejeunecysta sp.,dinoflagellates; palynology
9,Neogloboquadrina humerosa,benthic_forams; planktic_forams


In [154]:
len(df) == 11

True

In [155]:
df = check_taxa_by_groups(taxa_list_file, 'normalized_name')
df

,normalized_name,taxon_group
0,"""Amorphous organic matter""",dinoflagellates; palynology
1,"""Black phytoclasts""",dinoflagellates; palynology
2,"""Brown phytoclasts""",dinoflagellates; palynology
3,"""Fish teeth""",other; planktic_forams
4,"""Otoliths""",other; planktic_forams
5,Brigantedinium spp.,dinoflagellates; palynology
6,Bryozoa indet.,other; planktic_forams
7,Dinoflagellata indet.,dinoflagellates; nannofossils; palynology
8,Echinoidea indet.,other; planktic_forams
9,Foraminifera indet.,benthic_forams; dinoflagellates; palynology; p...


In [156]:
len(df) == 18

True

# check if taxon group matches file name

In [105]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral,update_group_abundances
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,True,False,True
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False,True
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False,True
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False,True
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False,False,False,False,False


In [106]:
taxa_df = pd.read_csv(taxa_crosswalk_list_file)

data = taxa_df.groupby(['verbatim_name'])['taxon_group'].apply('; '.join)
grouped_df = pd.DataFrame(data)
grouped_df.reset_index(inplace=True)

log_df(grouped_df)

(5368, 2)


,verbatim_name,taxon_group
0,"""Globigerina"" angulisuturalis _T",planktic_forams
1,"""Globigerina"" angulisuturalis _T_",planktic_forams
2,"""Globigerina"" ciperoensis _T",planktic_forams
3,"""Globigerina"" ciperoensis _T_",planktic_forams
4,"""Skeletonema"" utriculosa",diatoms


In [107]:
grouped_df[grouped_df['verbatim_name']=='Globigerinoides ruber (white)']

,verbatim_name,taxon_group
2164,Globigerinoides ruber (white),benthic_forams; planktic_forams


In [108]:
grouped_df[grouped_df['verbatim_name']=='Tenuitella gemma']

,verbatim_name,taxon_group
4764,Tenuitella gemma,planktic_forams


In [109]:
taxa_dict = {}
for index, row in grouped_df.iterrows():
    if row['verbatim_name'] not in taxa_dict:
        taxa_dict[row['verbatim_name']] = row['taxon_group']

In [110]:
taxa_dict['Globigerinoides ruber (white)']

'benthic_forams; planktic_forams'

In [111]:
taxa_dict['Tenuitella gemma']

'planktic_forams'

check if file taxon group is not in PI vetted taxon groups

In [112]:
data = []
all_taxa = taxa_df['verbatim_name']

for index, row in metadata.iterrows():
    # if index > 300:
    #     continue
        
    file_taxon_group = row['taxon_groups']

    df = pd.read_csv(CLEAN_DATA_DIR/row['path'])
    df = df.dropna(how='all', axis=1)

    taxa = set(df.columns).intersection(all_taxa)
    for taxon in taxa:
        if file_taxon_group not in taxa_dict[taxon] :
            name = taxa_df[taxa_df['verbatim_name'] == taxon]['normalized_name'].values[0]
            
            attr = {
                "verbatim_name": taxon, 
                "name": name,
                "file_taxon_group": file_taxon_group,
                "taxon_group": taxa_dict[taxon],
                "path": row['path']
            }
            data.append(attr)


In [113]:
report_df = pd.DataFrame(data)
report_df.sort_values(by=['name', 'taxon_group'], inplace=True)
report_df.shape

(219, 5)

In [114]:
cols = ['verbatim_name', 'name', 'file_taxon_group', 'taxon_group']
report_df.drop_duplicates(subset=cols, inplace=True)
log_df(report_df)

# 160

(160, 5)


,verbatim_name,name,file_taxon_group,taxon_group,path
118,Challengeria spp.,"""Challengeria spp.""",benthic_forams,radiolarians,LIMS/Micropal_CSV_2/356-U1462C_benthic_forams.csv
98,Acritarchs,Acritarcha indet.,palynology,dinoflagellates,LIMS/Micropal_CSV_2/374_U1522A_palynology.csv
95,Actiniscus pentasterias,Actiniscus pentasterias,diatoms,dinoflagellates,LIMS/Micropal_CSV_2/374_U1525A_diatoms.csv
39,Amaurolithus delicatus,Amaurolithus delicatus,diatoms,nannofossils,LIMS/Micropal_CSV_2/368_U1505D_diatoms.csv
27,Amaurolithus primus,Amaurolithus primus,diatoms,nannofossils,LIMS/Micropal_CSV_2/368_U1505D_diatoms.csv


In [115]:
len(report_df) == 160

True

check if file taxon group is not in PI vetted taxon groups when multiple taxon groups

In [116]:
data = []
all_taxa = taxa_df['verbatim_name']

for index, row in metadata.iterrows():
        
    file_taxon_group = row['taxon_groups']

    df = pd.read_csv(CLEAN_DATA_DIR/row['path'])
    df = df.dropna(how='all', axis=1)

    taxa = set(df.columns).intersection(all_taxa)
    for taxon in taxa:
        if file_taxon_group not in taxa_dict[taxon] and ';' in taxa_dict[taxon]:
            name = taxa_df[taxa_df['verbatim_name'] == taxon]['normalized_name'].values[0]
            
            attr = {
                "verbatim_name": taxon, 
                "name": name,
                "file_taxon_group": file_taxon_group,
                "taxon_group": taxa_dict[taxon],
                "path": row['path']
            }
            data.append(attr)

print(len(data))

0


In [117]:
len(data) == 0

True

# create file of all taxa samples that should be imported

In [95]:
taxa_df = pd.read_csv(taxa_crosswalk_list_file)
log_df(taxa_df)


(5380, 18)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments,eodp_id
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>100 m group,0
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>50 m group,1
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams,Others,NaN,not a taxa,Andy,other benthic foraminifera,2
3,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pleurostomellidae indet.,benthic_forams,Pleurostomellids comment,NaN,not a taxa,Andy,NaN,3
4,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ostracoda indet.,benthic_forams,Ostracoda spp.,Ostracoda spp.,group abundance,"Summer 2020: go in as Ostracoda indet, and the...",NaN,4


In [96]:
taxa_names = set(taxa_df['normalized_name'])
verbatim_names = set(taxa_df['verbatim_name'])

In [97]:
taxa = {}
for index, row in taxa_df.iterrows():
    taxa[row['verbatim_name']] = row['normalized_name']
    
len(taxa)

5368

In [99]:
data = []
for clean_data_path in clean_data_paths:
    
    raw_csvs = glob.glob(f"{clean_data_path}/*.csv")
    
    for csv in raw_csvs:
        # print(csv)
        df = pd.read_csv(csv, dtype=str)
        file_verbatim_names = set(df.columns).intersection(verbatim_names)
        for index, row in df.iterrows():
            for v_name in file_verbatim_names:
                if pd.notna(row[v_name]):
         
                    data.append({
                        'sample': row['Sample'], 
                         'verbatim_name': v_name, 
                         'name': taxa[v_name], 
                         'code': row[v_name], 
                         'path': csv.split('../../output/cleaned_data/')[1]
                    })

In [100]:
len(data)

312548

In [118]:
all_records = pd.DataFrame(data)
all_records.to_csv(OUTPUT_DIR/'tmp'/'all_taxa_samples.csv', index=False)

compare all_records with what was imported

In [158]:
sql = """
CREATE TABLE tmp_samples_taxa (
    id SERIAL PRIMARY KEY,
    sample character varying,
    verbatim_name character varying,
    name character varying,
    code character varying,
    path text
);
"""


sql = """ 
select sample, verbatim_name, name, code, path from tmp_samples_taxa

except

select samples.name as sample_name, taxa_crosswalk.verbatim_name, 
taxa.name,  samples_taxa.code, 
samples_taxa.data_source_notes as path
from samples_taxa
join samples on samples.id = samples_taxa.sample_id
join taxa on taxa.id = samples_taxa.taxon_id
join taxa_crosswalk on taxa_crosswalk.id = samples_taxa.original_taxon_id
;
"""

# create file of all field samples that should be imported

In [12]:
nontaxa_df = pd.read_csv(nontaxa_fields_path)
log_df(nontaxa_df)

(144, 4)


,original_name,name,dataset,dataset_type
0,Top Offset (cm) on Parent Sample,Top Offset (cm) on Parent Sample,LIMS,taxa
1,Bottom Offset (cm) on Parent Sample,Bottom Offset (cm) on Parent Sample,LIMS,taxa
2,mean depth (mbsf),mean depth (mbsf),LIMS,taxa
3,(taxon names are in the header),Taxon Name,LIMS,taxa
4,Abundance,Group abundance,LIMS,taxa


In [13]:
original_name = set(nontaxa_df['original_name'])
name = set(nontaxa_df['name'])

In [14]:
nontaxa = {}
for index, row in nontaxa_df.iterrows():
    nontaxa[row['original_name']] = row['name']
    
len(nontaxa)

131

In [15]:
data = []
for clean_data_path in clean_data_paths:
    
    raw_csvs = glob.glob(f"{clean_data_path}/*.csv")
    
    for csv in raw_csvs:
        # print(csv)
        df = pd.read_csv(csv, dtype=str)
        file_nontaxa = set(df.columns).intersection(nontaxa)
        for index, row in df.iterrows():
            for nt in file_nontaxa:
                if pd.notna(row[nt]):
         
                    data.append({
                        'sample': row['Sample'], 
                         'original_name': nt, 
                         'name': nontaxa[nt].strip(), 
                         'code': row[nt], 
                         'path': csv.split('../../output/cleaned_data/')[1]
                    })

In [16]:
all_records = pd.DataFrame(data)
log_df(all_records)

(104645, 5)


,sample,original_name,name,code,path
0,363-U1482A-1H-CC,Percentage of non-calcareous agglutinated fora...,Percentage of non-calcareous agglutinated fora...,1,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv
1,363-U1482A-1H-CC,Group abundance,Group abundance,R,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv
2,363-U1482A-1H-CC,Preservation,Preservation,E,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv
3,363-U1482A-2H-CC,Percentage of non-calcareous agglutinated fora...,Percentage of non-calcareous agglutinated fora...,1,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv
4,363-U1482A-2H-CC,Group abundance,Group abundance,R,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv


In [17]:
all_records.to_csv(OUTPUT_DIR/'tmp'/'all_field_samples.csv', index=False)

compare all_records with what was imported

In [ ]:
sql = """
CREATE TABLE tmp_samples_fields (
    sample character varying,
    original_name character varying,
    name character varying,
    code character varying,
    path character varying
);

"""

sql = """ 
select * from tmp_samples_fields

except

select samples.name as sample_name, samples_fields.original_name, 
fields.name,  samples_fields.value, 
samples_fields.data_source_notes as path
from samples_fields
 join fields on fields.id = samples_fields.field_id
 join samples on samples.id = samples_fields.sample_id;
;
"""

## check abundances codes

In [4]:
codes_df = pd.read_csv(PI_abundance_codes_group_path, dtype=str)
log_df(codes_df)

(2085, 8)


,Exp,original_header,taxon_group,abundance_code,preservation in andance column,harmonized_code,abundance_preservation_definition,file
0,320,% Planktic Foraminifera within whole sample,planktic_forams,0,NaN,0,% Planktic Foraminifera within whole sample,320_U1331B_Planktic_Forams.csv; 320_U1331B_Pla...
1,321,% Planktic Foraminifera within whole sample,planktic_forams,0,NaN,0,% Planktic Foraminifera within whole sample,321_U1338A_Planktic_forams.csv; 321_U1338A_Pla...
2,321,% Planktic Foraminifera within whole sample,planktic_forams,0,NaN,0,% Planktic Foraminifera within whole sample,321_U1337A_Planktic_forams.csv; 321_U1337A_Pla...
3,321,% Planktic Foraminifera within whole sample,planktic_forams,0.5,NaN,0.5,% Planktic Foraminifera within whole sample,321_U1337A_Planktic_forams.csv
4,320,% Planktic Foraminifera within whole sample,planktic_forams,1,NaN,1,% Planktic Foraminifera within whole sample,320_U1332A_Planktic_Forams.csv


In [5]:
sql = """select  fields.name, value , samples_fields.data_source_notes,
samples_fields.original_name
from fields 
join samples_fields on fields.id = samples_fields.field_id


group by fields.name, value , samples_fields.data_source_notes,
samples_fields.original_name
"""

rows = db.fetch_all(sql)
data = []
for row in rows:
    data.append({
        'name': row['name'],
        'value': row['value'],
        'data_source_notes': row['data_source_notes'],
        "original_name": row['original_name']
    })
    

db_df = pd.DataFrame(data)
log_df(db_df)

(19502, 4)


,name,value,data_source_notes,original_name
0,Age Control Comments,(NO OCEANICA),LIMS/Micropal_CSV_2/350_U1437B_nannofossils.csv,Zone comment
1,Age Control Comments,0.61-0.75,LIMS/Micropal_CSV_2/350_U1436A_planktic_forams...,Datum comment
2,Age Control Comments,0.61-0.75,LIMS/Micropal_CSV_2/350_U1436A_planktic_forams...,Zone comment
3,Age Control Comments,0.61-0.75 Ma based on co-occurence of G. hessi...,LIMS/Micropal_CSV_2/350_U1437B_planktic_forams...,Datum comment
4,Age Control Comments,0.61-0.75 Ma based on co-occurence of G. hessi...,LIMS/Micropal_CSV_2/350_U1437B_planktic_forams...,Zone comment


In [6]:
filter_codes = codes_df[codes_df['original_header'].isin(db_df['original_name'])].copy()



In [7]:
def unique_values(value):
    return list(set(value.split('; ')))

filter_codes['files'] = filter_codes['file'].apply(unique_values)

log_df(filter_codes)

(2085, 9)


,Exp,original_header,taxon_group,abundance_code,preservation in andance column,harmonized_code,abundance_preservation_definition,file,files
0,320,% Planktic Foraminifera within whole sample,planktic_forams,0,NaN,0,% Planktic Foraminifera within whole sample,320_U1331B_Planktic_Forams.csv; 320_U1331B_Pla...,"[320_U1331B_Planktic_Forams.csv, 320_U1332A_Pl..."
1,321,% Planktic Foraminifera within whole sample,planktic_forams,0,NaN,0,% Planktic Foraminifera within whole sample,321_U1338A_Planktic_forams.csv; 321_U1338A_Pla...,[321_U1338A_Planktic_forams.csv]
2,321,% Planktic Foraminifera within whole sample,planktic_forams,0,NaN,0,% Planktic Foraminifera within whole sample,321_U1337A_Planktic_forams.csv; 321_U1337A_Pla...,[321_U1337A_Planktic_forams.csv]
3,321,% Planktic Foraminifera within whole sample,planktic_forams,0.5,NaN,0.5,% Planktic Foraminifera within whole sample,321_U1337A_Planktic_forams.csv,[321_U1337A_Planktic_forams.csv]
4,320,% Planktic Foraminifera within whole sample,planktic_forams,1,NaN,1,% Planktic Foraminifera within whole sample,320_U1332A_Planktic_Forams.csv,[320_U1332A_Planktic_Forams.csv]


In [8]:
set(filter_codes['harmonized_code']) - set(db_df['value']) 

{nan}

In [9]:
data = []
for index, row in filter_codes.iterrows():
    if pd.isna(row['harmonized_code']):
        continue 
        
    for file in  row['files']:
        tmp = db_df[
            db_df['data_source_notes'].str.contains(file) & 
            (db_df['original_name'] == row['original_header']) &
            (db_df['value'] == row['harmonized_code'])
        ]
        if len(tmp) != 1:
            tmp = {'file': file}
            for col in ['original_header', 'harmonized_code', 'preservation in andance column']:
                tmp[col] = row[col]
                
            data.append(tmp)



In [10]:
new_df = pd.DataFrame(data)
log_df(new_df)

(132, 4)


,file,original_header,harmonized_code,preservation in andance column
0,321_U1337A_Planktic_forams.csv,% Planktic Foraminifera within whole sample,0,NaN
1,321_U1337A_Planktic_forams.csv,% Planktic Foraminifera within whole sample,2,NaN
2,321_U1337A_Planktic_forams.csv,% Planktic Foraminifera within whole sample,3,NaN
3,321_U1337A_Planktic_forams.csv,% Planktic Foraminifera within whole sample,4,NaN
4,321_U1337A_Planktic_forams.csv,% Planktic Foraminifera within whole sample,7,NaN


In [11]:
new_df.to_csv(OUTPUT_DIR/'tmp'/'mismatch_abundances.csv')